### Import Necessary Libraries

In [1]:
import numpy as np
import pandas as pd
import string  
import re
from tqdm.notebook import tqdm

### Load and Clean Dataset

In [2]:
# Load word set
word_path = './words.txt'
word_file = open(word_path, 'r')
words = word_file.readlines()

In [3]:
print(f'Total Number of Words in Dataset: {len(words)}')

Total Number of Words in Dataset: 227300


In [4]:
# Clean word set by removing whitespace, converting to lowercase, removing duplicates, and removing words with numbers
words = [w.strip('\n').lower() for w in words]
words = list(set(words))
words = [w for w in words if w.isalpha()]

In [5]:
print(f'Total Number of Words in Dataset After Cleaning: {len(words)}')

Total Number of Words in Dataset After Cleaning: 227300


In [6]:
# Split word set into training and validation
train_idx = np.random.choice(len(words), len(words) // 2, replace=False)
val_idx = np.array(list(set(range(len(words))) - set(train_idx)))

train_words = list(np.array(words)[train_idx])
val_words = list(np.array(words)[val_idx])

In [7]:
print(f'Total Number of Words in Training Dataset: {len(train_words)}')
print(f'Total Number of Words in Training Dataset: {len(val_words)}')

Total Number of Words in Training Dataset: 113650
Total Number of Words in Training Dataset: 113650


### Define Helper Functions

In [8]:
def letter2index(letter):
    return ord(letter) - ord('a')

In [9]:
def index2letter(ind):
    return chr(ind + ord('a'))

In [10]:
class HangmanGame():
    
    def __init__(self, corpus, N_LIVES=6):

        self.corpus = corpus
        self.N_LIVES = N_LIVES

    def word2info(self):

        info = []

        for letter in self.word:
            if letter in self.guessed:
                info.append(letter)
            else:
                info.append('_')

        return info

    def start(self, verbose=False):

        self.word = np.random.choice(self.corpus, 1)[0]
        self.unused = set(string.ascii_lowercase)
        self.guessed = set()
        self.info = self.word2info()
        self.LIVES_LEFT = self.N_LIVES
        self.ongoing = True
        self.verbose = verbose

        if self.verbose:
            print(self.info)
            print(f'LIVES LEFT: {self.LIVES_LEFT}')

        return self.info

    def guess(self, letter):

        if not self.ongoing:
            
            if self.verbose:
                print('The game is already over! Stop making guesses!')
            return -2
        
        elif letter in self.guessed:
            
            if self.verbose:
                print('You already guessed this letter! Try another letter!')
            return 0
        
        else:

            update_set = set([letter])
            self.guessed.update(update_set)
            self.unused = self.unused - update_set
            self.info = self.word2info()

            if letter not in self.word:
                self.LIVES_LEFT -= 1

            if self.LIVES_LEFT == 0:

                if self.verbose:
                    print(f'You lose! The word was {self.word}.')
                return -1
            elif ''.join(self.info) == self.word:
                if self.verbose:
                    print('You win!')
                return 2
            else:
                if self.verbose:
                    print(self.info)
                    print(f'LIVES LEFT: {self.LIVES_LEFT}')
                return self.info

In [11]:
def count_ngrams(corpus, n):

    ngrams = np.zeros((26,) * n)

    for word in corpus:
        for i in range(0, len(word)-n+1):

            indices = np.zeros(n, dtype='int')

            for j in range(n):
                indices[j] = letter2index(word[i+j])

            indices = tuple(indices)

            ngrams[indices] += 1
            
    return ngrams

In [12]:
def mask_vec(vec, guessed, mask_val=0):

    vec = vec.copy()

    if len(guessed) > 0:

        for letter in guessed:
            vec[letter2index(letter)] = 0

    return vec

In [13]:
def get_window(lst, index, width=1):

    start_width = 0
    for i in range(1, 2*width + 1):
        if (index - i >= 0) and (lst[index - i] != '_'):
            start_width += 1
        else:
            break

    end_width = 0
    for i in range(1, 2*width + 1):
        if (index + i < len(lst)) and (lst[index + i] != '_'):
            end_width += 1
        else:
            break

    if (end_width >= width) and (start_width >= width):
        end_width = width
        start_width = width
    elif end_width < width:
        start_width = min(2*width - end_width, start_width)
    elif start_width < width:
        end_width = min(2*width - start_width, end_width)

    start = index - start_width
    end = index + end_width

    sublist = lst[start:end + 1]
    new_index = index - start
    return sublist, new_index

In [14]:
def make_guess(grams, info, guessed, max_width=1):

    probs = np.zeros(26)

    n = len(info)

    for i in range(n):

        if info[i] == '_':

            window, center = get_window(info, i, width=max_width) 
            m = len(window)

            vec = grams[m-1].copy()
            sub = 0

            for j in range(m):
                if j != center:
                    vec = np.take(vec, letter2index(window[j]), axis=j-sub)
                    sub += 1               

            vec = mask_vec(vec, guessed)

            while vec.sum() == 0:

                vec = mask_vec(grams[0], guessed)
            
            probs += vec / vec.sum()
    
    probs = mask_vec(probs, guessed, mask_val=-1)
    return index2letter(np.argmax(probs))



In [15]:
def info2regex(info, unused):

    regex = r'^'
    unknown = rf'[{''.join(list(unused))}]'

    for space in info:
        if space == '_':
            regex = regex + unknown
        else:
            regex = regex + rf'{space}'

    regex = regex + r'$'

    return regex

In [16]:
vowels = 'a e i o u'.split()

In [17]:
def vowel_percentage(word):
    vowel_count = sum(1 for char in word if char in vowels)
    total_letters = len(word)
    if total_letters == 0:
        percentage = 0
    else:
        percentage = (vowel_count / total_letters) * 100
    return percentage

### Run Trials for Algorithm

In [18]:
N_TRIALS = 1000
game = HangmanGame(val_words)

In [19]:
full_grams = []
max_width = 2
    
for i in range(1, 2*max_width + 2):
    full_grams.append(count_ngrams(train_words, i))

In [20]:
wins = 0

In [21]:
for trial in tqdm(range(N_TRIALS)):

    info = game.start()
    win = False
    unused = game.unused.copy()
    guessed = game.guessed.copy()
    unused_vowels = set(vowels)
    n_lives = game.LIVES_LEFT
    pattern = info2regex(info, unused)
    matches = [word for word in train_words if re.fullmatch(pattern, word)]
    subcorpus = matches.copy()

    m = len(info)
    vps = [vowel_percentage(word) for word in train_words if len(word) <= m]
    threshold = np.quantile(np.array(vps), q=0.75)
    
    while (n_lives > 0) and (win == False):
    
        guess = make_guess(full_grams, info, guessed, max_width=max_width)

        output = game.guess(guess)

        if type(output) == list:
            info = output
        elif output == 2:
            win = True

        update_set = set([guess])
        guessed.update(update_set)
        unused = unused - update_set
        unused_vowels = unused_vowels - update_set

        vp = vowel_percentage(info)
        
        if (vp >= threshold):
            unused = unused - set(unused_vowels)
            guessed.update(set(unused_vowels))
        
        
        n_lives = game.LIVES_LEFT
        pattern = info2regex(info, unused)
        matches = [word for word in matches if re.fullmatch(pattern, word)]

    if win:
        wins += 1


  0%|          | 0/1000 [00:00<?, ?it/s]

In [22]:
print(f'Win Percentage on Validation Set: {wins / N_TRIALS}')

Win Percentage on Validation Set: 0.502
